In [61]:
from collections import UserDict
from numbers import Integral

In [62]:
test_cells = {
        (0, 0): ' ',
        (0, 1): ' ',
        (0, 2): ' ',
        (1, 0): ' ',
        (1, 1): 'a',
        (1, 2): ' ',
        (2, 0): ' ',
        (2, 1): ' ',
        (2, 2): ' ',
    }
list(test_cells.keys())


[(0, 1), (1, 2), (0, 0), (2, 0), (1, 0), (2, 2), (0, 2), (2, 1), (1, 1)]

In [94]:
class CellSpace(UserDict):
    
    @property
    def width(self):
        return max(list(self.data.keys()), key=lambda t: t[0])[0]
    
    @property
    def height(self):
        return max(list(self.data.keys()), key=lambda t: t[1])[1]
    
    def __getitem__(self, index):
        cls = type(self)
        new_cells = cls()
        width = self.width
        height = self.height
        if isinstance(index, tuple) and isinstance(index[0], slice) and isinstance(index[1], slice):
            # handle the case where both items are slices aka box case
            x_s = index[0].indices(width)
            y_s = index[1].indices(height)
            for y_index, y in enumerate(range(*y_s)):
                for x_index, x in enumerate(range(*x_s)):
                    new_cells[(x_index, y_index)] = self.data[(x, y)]
            return new_cells
        
        elif isinstance(index, tuple) and isinstance(index[0], slice) and isinstance(index[1], Integral):
            # handle the case where x is a slice, but y is an integer [int:int, int] aka row case
            x_s = index[0].indices(width)
            for x_index, x in enumerate(range(*x_s)):
                new_cells[(x_index, 0)] = self.data[(x, index[1])]
            return new_cells
        
        elif isinstance(index, tuple) and isinstance(index[0], Integral) and isinstance(index[1], slice):
            # handle the case where x is an integer, but y is a slice [int, int:int] aka column case
            y_s = index[1].indices(height)
            for y_index, y in enumerate(range(*y_s)):
                new_cells[(0, y_index)] = self.data[(index[0], y)]
            return new_cells
        
        elif isinstance(index, tuple) and isinstance(index[0], Integral) and isinstance(index[1], Integral):
            # handle the case where both items are integers [int,int] aka cell case
            return self.data[(index[0], index[1])]
        
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

In [95]:
c = CellSpace()
for x,y in [(x,y) for y in range(10) for x in range(10)]:
    c[(x,y)] = x+y

In [97]:
print(c[3:4,3:4])
print(c[(3,3)])
print(c[2,:])
print(type(c[:,:]))
print('width = {}, height = {}'.format(c.width, c.height))

{(0, 0): 6}
6
{(0, 1): 3, (0, 6): 8, (0, 0): 2, (0, 5): 7, (0, 4): 6, (0, 3): 5, (0, 8): 10, (0, 2): 4, (0, 7): 9}
<class '__main__.CellSpace'>
width = 9, height = 9
